In [3]:
import folium

# Coordinates for Bangladesh
bd_coordinates = (23.685, 90.356)

# Create a map centered around Bangladesh with OpenStreetMap tileset
bd_map = folium.Map(location=bd_coordinates, zoom_start=7, tiles='OpenStreetMap')

# Custom icon for the bridges
icon = folium.Icon(color="blue", icon="info-sign")

# Coordinates of important bridges in Bangladesh
bridges = {
    "Padma Bridge": (23.8251, 90.3686),
    "Bangabandhu Bridge": (24.0794, 89.6529),
    "Jamuna Bridge": (24.4366, 89.8930),
    "Meghna Bridge": (23.0238, 90.7287)
}

# Add markers for each bridge with customized icons
for bridge, coords in bridges.items():
    folium.Marker(location=coords, popup=bridge, icon=icon).add_to(bd_map)

# Display the map
bd_map



In [2]:
print('Hello world')

Hello world


In [3]:
print('test')

test
